In code part named "Sales" I want to analysis sales by best and worst products, sales in time or online sales. 

In [ ]:
--Sales--

SELECT sod.SalesOrderID,
        SalesOrderDetailID,
        OrderQty,
        sod. ProductID,
        p.Name 'Product',
        UnitPrice,
        LineTotal,
        st.Name 'Region',
        soh.OrderDate,
        soh.OnlineOrderFlag,
        ps.Name 'Subcategory'
from Sales.SalesOrderDetail sod 
JOIN sales.SalesOrderHeader soh 
    on soh.SalesOrderID = sod.SalesOrderID
JOIN Sales.SalesTerritory st 
    on soh.TerritoryID = st.TerritoryID
JOIN Production.Product p 
    on p.ProductID = sod.ProductID
join Production.ProductSubcategory ps ON
    ps.ProductSubcategoryID = p.ProductSubcategoryID


In the second code I have to add colums to get predicted sales and predicted costs. If quantity is bigger than safety stock level we have predicted sales, but if it lower than that we have to product more stuff to reach safety level and gains costs.

In [ ]:
--Warehouse--

SELECT pi.*, l.Name 'Warehouse',
l.LocationID, 
pc.Name 'Category_Name', 
p.Name 'Product_Name', 
p.SafetyStockLevel, 
p.StandardCost, 
p.ListPrice,
case
when pi.Quantity > p.SafetyStockLevel 
    then (pi.Quantity - p.SafetyStockLevel) * p.ListPrice
    else 0
    end as 'Predicted_Sales',
case 
when pi.Quantity < p.SafetyStockLevel
    then (p.SafetyStockLevel - pi.Quantity) * p.StandardCost
    else 0
    end as 'Predicted_Costs'
FROM Production.ProductInventory pi 
JOIN Production.[Location] l 
    on l.LocationID = pi.LocationID
JOIN Production.Product p 
    on p.ProductID = pi.ProductID
JOIN Production.ProductCategory pc 
    on pc.ProductCategoryID = p.ProductSubcategoryID




Third one contains two tables contains retail and individual customers. They are connected with union all.

In [ ]:
--Clients--

SELECT s.BusinessEntityID 'CustomerID'
    ,s.Name 'CustomerName'
    ,a.AddressLine1
    ,a.City
    ,sp.Name 'StateProvince'
    ,cr.Name 'Country'
    ,soh.TotalDue
    ,'Retail' 'CustomerType'
FROM Sales.Store s
JOIN Sales.Customer c 
    on c.StoreID = s.BusinessEntityID
JOIN Person.BusinessEntityAddress bea 
    on bea.BusinessEntityID = s.BusinessEntityID
join Person.Address a 
    on a.AddressID = bea.AddressID
JOIN Person.StateProvince sp 
    on sp.StateProvinceID = a.StateProvinceID
join Person.CountryRegion cr 
    on cr.CountryRegionCode = sp.CountryRegionCode
JOIN Sales.SalesOrderHeader soh 
    on soh.CustomerID = c.CustomerID
union all 
SELECT c.CustomerID
    ,CONCAT_WS(' ',p.FirstName,p.LastName) 'CustomerName'
    ,a.AddressLine1
    ,a.City
    ,sp.Name 'StateProvince'
    ,cr.Name 'Country'
    ,soh.TotalDue
    ,'Individual' 'CustomerType'
FROM Sales.Customer c
JOIN Person.Person p
    on p.BusinessEntityID = c.PersonID
JOIN Person.BusinessEntityAddress bea 
    on bea.BusinessEntityID = p.BusinessEntityID
join Person.Address a 
    on a.AddressID = bea.AddressID
JOIN Person.StateProvince sp 
    on sp.StateProvinceID = a.StateProvinceID
join Person.CountryRegion cr 
    on cr.CountryRegionCode = sp.CountryRegionCode
JOIN Sales.SalesOrderHeader soh 
    on soh.CustomerID = c.CustomerID



Production losses is another part of my analysis. As not to lose much data without defined product line, I changed non-defined lines to line 'L' to better analysis. 

In [ ]:
--Production losses--

SELECT p.ProductID,
        p.Name,
        p.SafetyStockLevel,
        p.ReorderPoint,
        p.StandardCost,
        p.ListPrice,
        p.DaysToManufacture,
        p.ProductLine,
        pc.Name 'Category',
        ps.Name 'Subcategory',
        wo.ScrappedQty,
        wo.ScrappedQty * p.StandardCost 'ProductionLosses',
        wo.ScrappedQty * p.ListPrice 'LoosedIncomes',
case 
    when p.ProductLine is null
    then 'L'
    else p.ProductLine
    end as ProductLine2 
from Production.[Product] p 
join Production.ProductSubcategory ps ON
    ps.ProductSubcategoryID = p.ProductSubcategoryID
join Production.WorkOrder wo ON
    wo.ProductID = p.ProductID
join Production.ProductCategory pc ON
    pc.ProductCategoryID = ps.ProductCategoryID




The last code in my project was about employees. I want to show all necessary information about company employees.

In [ ]:
--Employees--

select p.BusinessEntityID
    ,p.PersonType
    ,CONCAT_WS(' ',p.FirstName,p.MiddleName,p.LastName) Employee
    ,e.Gender
    ,d.Name Department
    ,e.JobTitle
    ,pp.PhoneNumber
    ,a.City
    ,CONCAT_WS(' ',a.AddressLine1,a.AddressLine2) Address
    ,sp.Name StateProvince 
    ,cr.Name Country
from Person.Person p 
JOIN HumanResources.EmployeeDepartmentHistory edh 
    on edh.BusinessEntityID = p.BusinessEntityID
JOIN HumanResources.Department d 
    on d.DepartmentID = edh.DepartmentID
join HumanResources.Employee e 
    on e.BusinessEntityID = p.BusinessEntityID
join Person.PersonPhone pp
    on pp.BusinessEntityID = p.BusinessEntityID
JOIN Person.BusinessEntityAddress bea 
    on bea.BusinessEntityID = p.BusinessEntityID
join Person.Address a 
    on a.AddressID = bea.AddressID
join Person.StateProvince sp
    on sp.StateProvinceID = a.StateProvinceID
join Person.CountryRegion cr 
    on cr.CountryRegionCode = sp.CountryRegionCode
order by p.BusinessEntityID